In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.show_dimensions", False)
pd.set_option("display.float_format", "{:4.2g}".format)

In [2]:
from IPython.core.magic import register_line_magic

@register_line_magic
def C(line):
    from IPython.core.getipython import get_ipython
    from fnmatch import fnmatch

    line = line.strip()
    if ' ' in line:
        idx_space = line.index(' ')
        space_num = line[:idx_space]
        if space_num.isdecimal():
            space_num = int(space_num)
            line = line[idx_space:]
        else:
            space_num = 5
    else:
        space_num = 5

    output_dict = {}
    cmds = line.split(';')
    for cmd in cmds:
        cmd = cmd.strip()
        if cmd != "":
            output_dict[cmd] = repr(eval(cmd)).split("\n")

    str_maxlen_in_cols = [max(len(cmd), len(max(data, key=len))) for cmd, data in output_dict.items()]
    data_row_max = max([len(v) for v in output_dict.values()])

    out_lines = [""]*(data_row_max+2)

    space=''
    for i, (cmd, data) in enumerate(output_dict.items()):
        w = str_maxlen_in_cols[i]

        out_lines[0]+=space+f'{cmd:^{w}}'
        out_lines[1]+=space+"-"*w
        for j, d in enumerate(data, 2):
            out_lines[j]+=space+f'{d:{w}}'

        if len(data) < data_row_max:
            for j in range(len(data)+2, data_row_max+2):
                out_lines[j]+=space+' '*w
        
        space = ' '*space_num

    for line in out_lines:
        print(line)

## 分群組運算

所謂分組運算是指使用特定的條件將資料分為多個分組，然後對每個分組進行運算，最後再將結果整合起來。Pandas 中的分組運算由 `DataFrame` 或 `Series` 物件的 `groupby()` 方法實現。

下面以某種藥劑的實驗資料 "dose.csv" 為例介紹如何使用分組運算分析資料。

在該資料集中使用了 "ABCD" 四種不同的藥劑處理方式(Tmt)，針對不同性別(Gender), 不同年齡(Age)的患者進行藥劑實驗，記錄下藥劑的投藥量(Dose)與兩種藥劑反應(Response)。

In [3]:
dose_df = pd.read_csv("dose.csv")
print( dose_df.head(3) )

   Dose  Response1  Response2 Tmt  Age Gender
0    50        9.9         10   C  60s      F
1    15      0.002      0.004   D  60s      F
2    25       0.63        0.8   C  50s      M


### `groupby()`方法

如圖 5-5 所示，分組操作中有關兩組資料：來源資料 和 分組資料。將 分組資料 傳遞給 來源資料 的 `groupby()` 方法以完成分組。`groupby()` 的 `axis` 參數預設為 0，表示對來源資料的行(row)進行分組。來源資料 中的每行與 分組資料 中的每個元素對應，分組資料 中的每個唯一值對應一個分組。由圖中的分組資料中有兩個唯一值，因此獲得兩個分組。

![groupby](groupby.png)

圖5-5 groupby() 分組示意圖

> **TIP**

> `groupby()`並不立即執行分群組動作，而只是傳回儲存源資料和分群組資料的`GroupBy`物件。在需要取得每個分群組的實際資料時，`GroupBy`物件才會執行分群組動作。

當分組用的資料在來源資料中時，可以直接透過 列名稱 指定分組資料。當來源資料是 `DataFrame` 型態時，`groupby()` 方法傳回一個 `DataFrameGroupBy` 物件。若來源資料是 `Series` 型態，則傳回 `SeriesGroupBy` 物件。在下面的實例中使用 Tmt 列對來源資料分組：

In [4]:
tmt_group = dose_df.groupby("Tmt")
print( type(tmt_group) )
print(tmt_group.head())

<class 'pandas.core.groupby.generic.DataFrameGroupBy'>
    Dose  Response1  Response2 Tmt  Age Gender
0     50        9.9         10   C  60s      F
1     15      0.002      0.004   D  60s      F
2     25       0.63        0.8   C  50s      M
3     25        1.4        1.6   C  60s      F
4     15       0.01       0.02   C  60s      F
5     20      0.007      0.079   D  50s      F
6      1          0          0   A  50s      F
7     20      0.038      0.033   C  60s      M
8     15      0.001      0.001   D  50s      F
9     40         11         10   B  60s      F
10    15        5.2        5.2   A  60s      F
11    15        1.4        1.7   B  50s      M
12     5          0      0.001   A  60s      F
13    40        9.7         10   B  60s      F
15    60        8.9          9   D  50s      M
16    30        9.8         10   B  60s      F
17     5          0      0.003   A  50s      M
19     1          0          0   B  60s      F
20 1e+02        9.9        8.8   D  50s      M
32 1e

還可以使用列表傳遞多組分組資料給 `groupby()` ，例如下面的程式使用處理方式與年齡對來源資料分組：

In [5]:
tmt_age_group = dose_df.groupby(["Tmt", "Age"])
print( tmt_age_group.head() )

     Dose  Response1  Response2 Tmt  Age Gender
0      50        9.9         10   C  60s      F
1      15      0.002      0.004   D  60s      F
2      25       0.63        0.8   C  50s      M
3      25        1.4        1.6   C  60s      F
4      15       0.01       0.02   C  60s      F
5      20      0.007      0.079   D  50s      F
6       1          0          0   A  50s      F
7      20      0.038      0.033   C  60s      M
8      15      0.001      0.001   D  50s      F
9      40         11         10   B  60s      F
10     15        5.2        5.2   A  60s      F
11     15        1.4        1.7   B  50s      M
12      5          0      0.001   A  60s      F
13     40        9.7         10   B  60s      F
14  1e+02        9.7         11   C  50s      M
15     60        8.9          9   D  50s      M
16     30        9.8         10   B  60s      F
17      5          0      0.003   A  50s      M
18     30        4.9        4.9   C  60s      F
19      1          0          0   B  60s

當分組資料不在來源資料中時，可以直接傳遞分組資料。在下面的實例中對長度與來源資料的行數相同、取值範圍為 `[0,5)` 的隨機整數陣列進行分組，這樣就將來源資料隨機分成了 5 組：

In [6]:
random_values = np.random.randint(0, 5, dose_df.shape[0])
random_group = dose_df.groupby(random_values)

當分組資料可以透過來源資料的行索引計算時，可以將計算函數傳遞給 `groupby()` ，下面的實例使用行索引值除以 3 的餘數進行分組，因此將來源資料的每行交替地分為 3 組。這是因為來源資料的行索引為從 0 開始的整數序列。

In [7]:
alternating_group = dose_df.groupby(lambda n:n % 3)

上述三種分組資料可以任意自由組合，例如下面的實例同時使用來源資料中的 性別列, 函數 以及 陣列進行分組：

In [8]:
crazy_group = dose_df.groupby(["Gender", lambda n: n % 2, random_values])

### `GroupBy`物件

使用 `len()` 可以取得分組數:

In [9]:
print( len(tmt_age_group), len(crazy_group) )

10 20


`GroupBy` 物件支援反覆運算介面，它與字典的 `iteritems()` 方法類似，每次反覆運算獲得分組的鍵和資料。當使用多列資料分組時，與每個組對應的鍵是一個元組：

In [10]:
for key, df in tmt_age_group:
    print( "key =", key, ", shape =", df.shape )

key = ('A', '50s') , shape = (39, 6)
key = ('A', '60s') , shape = (26, 6)
key = ('B', '40s') , shape = (13, 6)
key = ('B', '50s') , shape = (13, 6)
key = ('B', '60s') , shape = (39, 6)
key = ('C', '40s') , shape = (13, 6)
key = ('C', '50s') , shape = (13, 6)
key = ('C', '60s') , shape = (39, 6)
key = ('D', '50s') , shape = (52, 6)
key = ('D', '60s') , shape = (13, 6)


由於 python 的設定陳述式支援反覆運算介面，因此可以使用下面的敘述快速為每個分組資料指定變數名稱。這是因為我們知道只有 4 種藥劑處理方式，並且 `GroupBy` 物件預設會對分組鍵進行排序。可以將 `groupby()` 的 `sort` 參數設定為 `False` 以關閉排序功能，這樣可以稍微加強大量分組時的運算速度。

In [11]:
(_, df_A), (_, df_B), (_, df_C), (_, df_D) = tmt_group
%C df_A.head(); df_B.head(); df_C.head()

                 df_A.head()                                        df_B.head()                                        df_C.head()                 
----------------------------------------------     ----------------------------------------------     ---------------------------------------------
    Dose  Response1  Response2 Tmt  Age Gender         Dose  Response1  Response2 Tmt  Age Gender        Dose  Response1  Response2 Tmt  Age Gender
6      1          0          0   A  50s      F     9     40         11         10   B  60s      F     0    50        9.9         10   C  60s      F
10    15        5.2        5.2   A  60s      F     11    15        1.4        1.7   B  50s      M     2    25       0.63        0.8   C  50s      M
12     5          0      0.001   A  60s      F     13    40        9.7         10   B  60s      F     3    25        1.4        1.6   C  60s      F
17     5          0      0.003   A  50s      M     16    30        9.8         10   B  60s      F     4    15   

> **TIP**

> 由於`GroupBy`物件有`keys`屬性，因此無法透過`dict(tmt_group)`直接將其轉為字典，可以先將其轉為迭代器，再轉為字典`dict(iter(tmt_group))`。

`get_group()` 方法可以獲得與指定的分組鍵對應的資料，例如：

In [12]:
%C tmt_group.get_group("A").head(3);; tmt_age_group.get_group(("A", "50s")).head(3)

       tmt_group.get_group("A").head(3)            tmt_age_group.get_group(("A", "50s")).head(3) 
----------------------------------------------     ----------------------------------------------
    Dose  Response1  Response2 Tmt  Age Gender         Dose  Response1  Response2 Tmt  Age Gender
6      1          0          0   A  50s      F     6      1          0          0   A  50s      F
10    15        5.2        5.2   A  60s      F     17     5          0      0.003   A  50s      M
12     5          0      0.001   A  60s      F     34    40         11         10   A  50s      M


對 `GroupBy` 的索引操作將獲得一個只包含來源資料中指定列的新 `GroupBy` 物件，透過這種方式可以先使用來源資料中的某些列進行分組，然後選擇另一些列進行後續計算。

In [13]:
print( tmt_group["Dose"] )
print( tmt_group[["Response1", "Response2"]] )

`GroupBy` 類別中定義了 `__getattr__()` 方法，因此當取得 `GroupBy` 中未定義的屬性時，將按照下面的順序操作：
- 如果屬性名稱是來源資料物件的某列的名稱，則相當於 `GroupBy[name]`，即取得針對該列的 `GroupBy` 物件。
- 如果屬性名稱是來源資料物件的方法，則相當於透過 `apply()` 對每個分組呼叫該方法。注意 Pandas 中定義了轉為 `apply()` 的方法集合，只有在此集中之中的方法才會被自動轉換。關於 `apply()` 方法將在下一小節詳細介紹。

下面的程式獲得對來源資料中的 Dose 列進行分組的 `GroupBy` 物件：

In [14]:
print( tmt_group.Dose )

### 分群組－運算－合並

透過 `GroupBy` 物件提供的 `agg()`, `transform()`, `filter()`, `apply()` 等方法可以實現各種分組運算。每個方法的第一個參數都是一個回呼函數，該函數對每個分組的資料進行運算並傳回結果。這些方法根據回呼函數的傳回結果產生最後的分組運算結果。

#### `agg()`－聚合

`agg()` 對每個分組中的資料進行聚合運算。所謂聚合運算是指將一組由 N 個數值組成的資料轉為單一數值的運算，例如求和、平均值、中間值甚至隨機取值等都是聚合運算。其回呼函數接收的資料是表示每個分組中每列資料的 `Series` 物件，若回呼函數不能處理 `Series` 物件，則 `agg()` 會接著嘗試將整個分組的資料作為 `DataFrame` 物件傳遞給回呼函數。回呼函數對其參數進行聚合運算，將 `Series` 物件轉為單一數值，或將 `DataFrame` 物件轉為 `Series` 物件。`agg()` 傳回一個 `DataFrame` 物件，其行索引為每個分組的鍵，而列索引為來源資料的列索引。

![agg_transform](agg_transform.png)

圖 5-6 agg() 和 transform() 的運算示意圖

在圖 5-6 中，上方兩個表格顯示了兩個分組中的資料，而下方左側兩個表格顯示了對這兩個分組執行聚合運算之後的結果。其中最左側的表格是執行 `g.agg(np.max)` 的結果。由於 `np.max()` 能對 `Series` 物件進行運算，因此 `agg()` 將 分組a 和 分組b 中的每列資料分別傳遞給 `np.max()` 以計算每列的最大值，並將所有最大值聚合成一個 `DataFrame` 物件。例如分組a 中的 B 列傳遞給 `np.max()` 的計算結果為 6，該數值儲存在結果的第 a 行、第 B 列中。

左側第二個表格對應的程式為：

```
g.agg(lambda df: df.loc[(df.A + df.B).idxmax()])
```

由於在回呼函數中存取了屬性 A 和 B ，這兩個屬性在表示每列資料的 `Series` 物件中不存在，因此傳遞 `Series` 物件回呼函數的嘗試失敗。於是 `agg()` 接下來嘗試將表示整個分組資料的 `DataFrame` 物件傳遞給回呼函數。該回呼函數每次傳回結果中的一行，例如圖中 分組b 對應的運算結果為第b行。該回呼函數傳回每個分組中 A+B 最大的那一行。

下面是對 tmt_group 進行聚合運算的實例。

❶ 計算每個分組中每列的平均值，注意結果中自動剔除了無法求平均值的字串列。

❷ 找到每個分組中 Response1 最大的那一行，由於回呼函數對表示整個分組的 `DataFrame` 進行運算，因此結果中包含了來源資料中的所有列。

In [37]:
agg_res1 = tmt_group.agg(np.mean) #❶
# agg_res2 = tmt_group.agg(lambda df:df.loc[df.Response1.idxmax()]) #❷
agg_res2 = tmt_group.apply(lambda df:df.loc[df.Response1.idxmax()]) #❷
%C 4 agg_res1; agg_res2

           agg_res1                                   agg_res2                    
-------------------------------    -----------------------------------------------
     Dose  Response1  Response2         Dose  Response1  Response2 Tmt  Age Gender
Tmt                                Tmt                                            
A      34        6.7        6.9    A      80         11         10   A  60s      F
B      34        5.6        5.5    B   1e+02         11         10   B  50s      M
C      34          4        4.1    C      60         10         11   C  50s      M
D      34        3.3        3.2    D      80         11        9.9   D  60s      F


#### `transform()`－轉換

`transform()` 對每個分組中的資料進行轉換運算。與 `agg()` 相同，首先嘗試將表示每列的 `Series` 物件傳遞給回呼函數，如果失敗，將表示整個分組的 `DataFrame` 物件傳遞給回呼函數。回呼函數的傳回結果與參數的形狀相同，`transform()` 將這些結果按照來源資料的順序合併在一起。

圖 5-6 的下方右側兩個表格為 `transform()` 的運算結果，它們對應的回呼函數分別對 `Series` 和 `DataFrame` 物件進行處理。注意這兩個表格的 行索引 與 來源資料 的 行索引 相同。

下面是對 tmt_group 進行轉換運算的實例。
- ❶ 回呼函數能對 `Series` 物件進行運算，因此運算結果中不包含來源資料中的字串列。
- ❷ 由於 `Series` 物件沒有 `assign()` 方法，因此 `transform()` 在嘗試 `Series` 失敗之後，將表示整個分組的 `DataFrame` 物件傳遞給回呼函數。該回呼函數只對 Response1 列進行轉換。

In [39]:
transform_res1 = tmt_group.transform(lambda s:s - s.mean()) #❶
# transform_res2 = tmt_group.transform(
#     lambda df:df.assign(Response1=df.Response1 - df.Response1.mean())) #❷
transform_res2 = tmt_group.apply(
    lambda df:df.assign(Response1=df.Response1 - df.Response1.mean())) #❷
%C transform_res1.head(5); transform_res2.head(5)

   transform_res1.head(5)                    transform_res2.head(5)            
-----------------------------     ---------------------------------------------
   Dose  Response1  Response2        Dose  Response1  Response2 Tmt  Age Gender
0    16        5.8        5.9     0    50        5.8         10   C  60s      F
1   -19       -3.3       -3.2     1    15       -3.3      0.004   D  60s      F
2  -8.5       -3.4       -3.3     2    25       -3.4        0.8   C  50s      M
3  -8.5       -2.7       -2.6     3    25       -2.7        1.6   C  60s      F
4   -19         -4       -4.1     4    15         -4       0.02   C  60s      F


<ipython-input-39-5acaab5c67ae>:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.transform is deprecated. In a future version, a TypeError will be raised. Before calling .transform, select only columns which should be valid for the function.
  transform_res1 = tmt_group.transform(lambda s:s - s.mean()) #❶


#### `filter()`－過濾

`filter()` 對每個分組進行條件判斷。它將表示每個分組的 `DataFrame` 物件傳遞給回呼函數，該函數傳回 True 或 False，以決定是否保留該分組。`filter()` 的傳回結果是過濾掉一些行之後的 `DataFrame` 物件，其行索引與來源資料的行索引的順序一致。

下面的程式保留 Response1 列的最大值小於 11 的分組，注意結果中包含用於分組的列 Tmt：

In [40]:
print( tmt_group.filter(lambda df:df.Response1.max() < 11).head() )

   Dose  Response1  Response2 Tmt  Age Gender
0    50        9.9         10   C  60s      F
1    15      0.002      0.004   D  60s      F
2    25       0.63        0.8   C  50s      M
3    25        1.4        1.6   C  60s      F
4    15       0.01       0.02   C  60s      F


#### `apply()`－運用

`apply()` 將表示每個分組的 `DataFrame` 物件傳遞給回呼函數並收集其傳回值，並將這些傳回值按照某種規則合併。

`apply()` 的用法十分靈活，可以實現上述 `agg()`, `transform()`, `filter()` 方法的功能。它會根據回呼函數的傳回值的型態選擇恰當的合併方式，然而這種自動選擇有時會獲得令人費解的結果。

![apply](apply.png)

圖 5-7 apply() 的運算示意圖

圖 5-7 顯示了對包含 a 和 b 兩個分組的 `GroupBy` 物件 g 執行 `apply()` 後獲得的 4 種結果：

如圖 5-7 (左一)所示，回呼函數為 `DataFrame.max` ，它計算 `DataFrame` 物件中每列的最大值，傳回一個以列名為索引的 `Series` 物件，因此對於所有的分組資料傳回的索引都是相同的。這種情況下 `apply()` 的結果與 `agg()` 相同，是一個以每個分組的鍵為行索引、以所有傳回物件的索引為列索引的 `DataFrame` 物件。

如圖 5-7 (左二)所示，當回呼函數傳回的 `Series` 物件的索引不是全部一致時，`apply()` 將這些 `Series` 物件沿垂直方向連接在一起，獲得一個多級索引的 `Series` 物件。多級索引由分組的鍵和每個 `Seroes` 物件的索引組成。

如圖 5-7 (左三)中，回呼函數傳回的是 `DataFrame` 物件，並且結果的行索引與參數的行索引是同一索引物件，即滿足以下條件：

```
df.index is (df - df.min()).index
```

則 `apply()` 傳回的 `DataFrame` 物件的行索引與來源資料的索引一致，這與 `filter()` 的結果相同。

在圖 5-7 (左四) 中，回呼函數的傳回結果與(圖 5-7 左三)相同，但由於使用 `[:]` 複製了整個資料，因此其傳回物件與參數的索引不是同一物件。這種情況下，將按照分組的順序況垂直方向將所有傳回結果連接在一起，獲得一個多級索引的 `DataFrame` 物件。多級索引由分組的鍵和每個 `DataFrame` 物件的索引組成。

> **WARNING**

> 注意目前的版本采用`is`判斷索引是否相同，很容易引起混淆，未來的版本可能會對這一點進行修改。

下面計算 tmt_group 的每個分組中每列的最大值和平均值。注意最大值的結果中包含字串列，而平均值的結果中不包含字串列：

In [41]:
%C 4 tmt_group.apply(pd.DataFrame.max); tmt_group.apply(pd.DataFrame.mean)

       tmt_group.apply(pd.DataFrame.max)           tmt_group.apply(pd.DataFrame.mean)
-----------------------------------------------    ----------------------------------
     Dose  Response1  Response2 Tmt  Age Gender         Dose  Response1  Response2   
Tmt                                                Tmt                               
A   1e+02         11         11   A  60s      M    A      34        6.7        6.9   
B   1e+02         11         10   B  60s      M    B      34        5.6        5.5   
C   1e+02         10         11   C  60s      M    C      34          4        4.1   
D   1e+02         11        9.9   D  60s      M    D      34        3.3        3.2   


<string>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.


下面的程式從每個分組的 Response1 列隨機取兩個數值。
- ❶ 由於 `sample()` 保留與數值對應的標籤，因此結果是一個多級標籤的 `Series` 物件。
- ❷ 對 `sample()` 的結果呼叫 `reset_index()` 方法，這樣所有傳回結果的標籤全部相同，因此獲得的結果是一個 `DataFrame` 物件，其每一行與一個分組對應。

In [42]:
sample_res1 = tmt_group.apply(lambda df:df.Response1.sample(2)) #❶
sample_res2 = tmt_group.apply(
    lambda df:df.Response1.sample(2).reset_index(drop=True)) #❷
%C 4 sample_res1; sample_res2

          sample_res1                  sample_res2    
-------------------------------    -------------------
Tmt                                Response1    0    1
A    227      0                    Tmt                
     161    5.3                    A           10   10
B    145      0                    B          3.8    0
     31     0.4                    C         0.35   10
C    201      0                    D          9.4    0
     162     10                                       
D    153      0                                       
     64       0                                       
Name: Response1, dtype: float64                       


當回呼函數的傳回值是 `DataFrame` 物件時，根據其 行(row)標籤 是否與參數物件的 行(row)標籤 為同一物件，會獲得不同的結果：

In [43]:
group = tmt_group[["Response1", "Response1"]]
apply_res1 = group.apply(lambda df:df - df.mean())
apply_res2 = group.apply(lambda df:(df - df.mean())[:])

%C 4 apply_res1.head(); apply_res2.head()

   apply_res1.head()          apply_res2.head()   
-----------------------    -----------------------
   Response1  Response1       Response1  Response1
0        5.8        5.8    0        5.8        5.8
1       -3.3       -3.3    1       -3.3       -3.3
2       -3.4       -3.4    2       -3.4       -3.4
3       -2.7       -2.7    3       -2.7       -2.7
4         -4         -4    4         -4         -4


當回呼函數傳回 `None` 時，將忽略該傳回值，因此可以實現 `filter()` 的功能。下面的程式從 Response1 的平均值大於 5 的分組中隨機取兩行資料：

In [44]:
print( tmt_group.apply(lambda df:None if df.Response1.mean() < 5 else df.sample(2)) )

         Dose  Response1  Response2 Tmt  Age Gender
Tmt                                                
A   173 1e+02         10         10   A  60s      F
    150 1e+02         10         11   A  50s      F
B   39     80        9.3         10   B  60s      F
    145   0.1          0          0   B  50s      M


Pandas 使用 `Cython` 對一些常用的聚合功能進行了最佳化處理，例如 `mean()`, `median()`, `var()` 等。此外，`GroupBy` 還自動將一些常用的 `DataFrame` 方法用 `apply()` 包裝。因此，透過 `GroupBy` 物件呼叫這些方法就相當於將這些方法作為回呼數傳遞給 `apply()`。

In [45]:
%C 4 tmt_group.mean(); tmt_group.quantile(q=0.75)

       tmt_group.mean()              tmt_group.quantile(q=0.75)   
-------------------------------    -------------------------------
     Dose  Response1  Response2         Dose  Response1  Response2
Tmt                                Tmt                            
A      34        6.7        6.9    A      50         10         10
B      34        5.6        5.5    B      50        9.8         10
C      34          4        4.1    C      50        9.6        9.6
D      34        3.3        3.2    D      50        8.9        8.4


<string>:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.quantile is deprecated. In a future version, a TypeError will be raised. Before calling .quantile, select only columns which should be valid for the function.
